<a href="https://colab.research.google.com/github/NinaNikolova/data_mining_project/blob/main/python_games2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mодел за прогнозиране на рейтинги на видео игри
## Mодел за линейна регресия, който предсказва рейтинги на видео игри

In [61]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
uploaded = files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving video_game_reviews.csv to video_game_reviews (3).csv


## Импортиране на библиотеки
- pandas и numpy - за работа с данни и числови операции
- matplotlib и seaborn - за визуализация на данните
- sklearn библиотеки - за създаване на модела за машинно обучение, предварителна обработка на данните и оценка на резултатите
##Зареждане на данните
- Зареждаме данните от CSV файл в променлива df (DataFrame). Тези данни съдържат информация за видео игри и техните рейтинги.
## Използва се Video Game Reviews and Ratings dataset:
https://www.kaggle.com/datasets/jahnavipaliwal/video-game-reviews-and-ratings

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('/content/video_game_reviews.csv')
df.head()

,Game Title,User Rating,Age Group Targeted,Price,Platform,Requires Special Device,Developer,Publisher,Release Year,Genre,Multiplayer,Game Length (Hours),Graphics Quality,Soundtrack Quality,Story Quality,User Review Text,Game Mode,Min Number of Players
0,Grand Theft Auto V,36.4,All Ages,41.41,PC,No,Game Freak,Innersloth,2015,Adventure,No,55.3,Medium,Average,Poor,"Solid game, but too many bugs.",Offline,1
1,The Sims 4,38.3,Adults,57.56,PC,No,Nintendo,Electronic Arts,2015,Shooter,Yes,34.6,Low,Poor,Poor,"Solid game, but too many bugs.",Offline,3
2,Minecraft,26.8,Teens,44.93,PC,Yes,Bungie,Capcom,2012,Adventure,Yes,13.9,Low,Good,Average,"Great game, but the graphics could be better.",Offline,5
3,Bioshock Infinite,38.4,All Ages,48.29,Mobile,Yes,Game Freak,Nintendo,2015,Sports,No,41.9,Medium,Good,Excellent,"Solid game, but the graphics could be better.",Online,4
4,Half-Life: Alyx,30.1,Adults,55.49,PlayStation,Yes,Game Freak,Epic Games,2022,RPG,Yes,13.2,High,Poor,Good,"Great game, but too many bugs.",Offline,1


##  Изследване на данните
Извършваме първоначален анализ на данните:
- Проверяваме размера на набора от данни (брой редове и колони)
- Разглеждаме типовете данни във всяка колона
- Извеждаме списък с имената на всички колони
- Показваме първите 5 реда от данните
- Проверяваме за липсващи стойности във всяка колона

In [63]:
print("Dataset shape:", df.shape)
print("\nData types:")
print(df.dtypes)
print("\nColumn names in the dataset:")
print(df.columns.tolist())
print("\nSample data:")
print(df.head())
print("\nMissing values:")
print(df.isnull().sum())

Dataset shape: (47774, 18)

Data types:
Game Title                  object
User Rating                float64
Age Group Targeted          object
Price                      float64
Platform                    object
Requires Special Device     object
Developer                   object
Publisher                   object
Release Year                 int64
Genre                       object
Multiplayer                 object
Game Length (Hours)        float64
Graphics Quality            object
Soundtrack Quality          object
Story Quality               object
User Review Text            object
Game Mode                   object
Min Number of Players        int64
dtype: object

Column names in the dataset:
['Game Title', 'User Rating', 'Age Group Targeted', 'Price', 'Platform', 'Requires Special Device', 'Developer', 'Publisher', 'Release Year', 'Genre', 'Multiplayer', 'Game Length (Hours)', 'Graphics Quality', 'Soundtrack Quality', 'Story Quality', 'User Review Text', 'Game Mode', 'Min 

## Идентифициране на целевата променлива
Тук търсим коя колона съдържа рейтингите на игрите:
- Създаваме списък с няколко възможни имена на колони, които биха могли да съдържат рейтинги
- Проверяваме дали някое от тези имена съществува в нашите данни
- Ако не намерим такава колона, използваме първата числова колона като целева променлива
- Ако няма числови колони, програмата прекратява изпълнението с грешка

In [66]:
possible_target_columns = ['User Rating', 'Rating', 'User Review', 'user_review', 'Review Score']
target_column = None
for col in possible_target_columns:
    if col in df.columns:
        target_column = col
        print(f"\nTarget column found: {target_column}")
        break

if target_column is None:
    print("\nCould not identify target column. Please check the column names.")
    print("\nFirst 5 rows of the dataset:")
    print(df.head())
    numeric_cols = df.select_dtypes(include=['number']).columns
    if len(numeric_cols) > 0:
        target_column = numeric_cols[0]
        print(f"\nUsing {target_column} as the target column.")
    else:
        raise ValueError("No numeric columns found for regression.")


Target column found: User Rating


## Предварителна обработка на данните
- Визуализираме разпределението на рейтингите с хистограма, за да разберем характеристиките на целевата променлива
- Премахваме редовете, където рейтингите липсват, тъй като не можем да обучим модела с липсващи стойности в целевата променлива

In [67]:

plt.figure(figsize=(10, 6))
sns.histplot(df[target_column].dropna(), kde=True)
plt.title(f'Distribution of {target_column}')
plt.savefig('target_distribution.png')
plt.close()

df = df.dropna(subset=[target_column])

## Избор на характеристики
- Разделяме колоните на категорични и числови
- За категоричните колони, избираме само тези с по-малко от 50 уникални стойности, за да избегнем "експлозия" при one-hot encoding
- Изключваме целевата променлива от характеристиките
- Извеждаме списъците с избраните характеристики

In [68]:
columns_to_drop = ['Developer', 'Publisher', 'Game Title']
df = df.drop(columns=columns_to_drop)
df.drop
categorical_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_features = []
for col in categorical_columns:
    if col != target_column and df[col].nunique() < 50:
        categorical_features.append(col)

numeric_features = df.select_dtypes(include=['number']).columns.tolist()
numeric_features = [col for col in numeric_features if col != target_column]

print(f"\nSelected numeric features: {numeric_features}")
print(f"Selected categorical features: {categorical_features}")


Selected numeric features: ['Price', 'Release Year', 'Game Length (Hours)', 'Min Number of Players']
Selected categorical features: ['Age Group Targeted', 'Platform', 'Requires Special Device', 'Genre', 'Multiplayer', 'Graphics Quality', 'Soundtrack Quality', 'Story Quality', 'User Review Text', 'Game Mode']


##Анализ на корелацията:
- Изчисляваме корелацията между числовите характеристики и рейтингите;
- Визуализираме топ 10 характеристики с най-висока абсолютна корелация
- Това ни помага да разберем кои фактори най-силно влияят на рейтингите на игрите

In [69]:
if len(numeric_features) > 0:
    correlations = df[numeric_features + [target_column]].corr()[target_column].sort_values(ascending=False)
    print("\nFeature correlations with target:")
    print(correlations)

    plt.figure(figsize=(10, 8))
    top_corr = correlations.drop(target_column).abs().sort_values(ascending=False).head(10)
    sns.barplot(x=top_corr.values, y=top_corr.index)
    plt.title(f'Top Correlations with {target_column}')
    plt.savefig('feature_correlations.png')
    plt.close()


Feature correlations with target:
User Rating              1.000000
Price                    0.760659
Game Length (Hours)      0.628743
Min Number of Players   -0.000365
Release Year            -0.003128
Name: User Rating, dtype: float64


##Създаване на конвейер за предварителна обработка
Създаваме конвейери за обработка на различните типове данни:

- За числовите характеристики: попълваме липсващите стойности със средната стойност и стандартизираме данните
- За категоричните характеристики: попълваме липсващите стойности с най-често срещаната стойност и прилагаме one-hot encoding

Обединяваме тези конвейери в един общ преработвач (preprocessor)

In [70]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

##Създаване на модела
Създаваме цялостен конвейер, който включва предварителната обработка и линейния регресионен модел
Това опростява процеса на обучение и предсказване

In [71]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

##Разделяне на данните
Разделяме данните на характеристики (X) и целева променлива (y)
Използваме функцията train_test_split за разделяне на данните на тренировъчен (80%) и тестов (20%) набор
Задаваме random_state=42 за възпроизводимост на резултатите
Извеждаме размера на получените набори от данни

In [ ]:
X = df[numeric_features + categorical_features]
y = df[target_column]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nTraining set size: {X_train.shape}")
print(f"Testing set size: {X_test.shape}")


Training set size: (38219, 14)
Testing set size: (9555, 14)


 ##Обучение на модела
 Обучаваме модела върху тренировъчните данни
Функцията fit() изпълнява както предварителната обработка, така и обучението на линейния регресионен модел

In [72]:
print("\nTraining the model...")
model.fit(X_train, y_train)


Training the model...


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Price', 'Release Year',
                                                   'Game Length (Hours)',
                                                   'Min Number of Players']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Age Group Targeted',
                                                   'Platform',
                                                   'Requires Special Device',
                                                   'Genre', 'Multiplayer',
                                                   'Graphics Quality',
                                                   'Soundtrack Quality',
                                                   'Story Quality',
                                                   'User Review Text',
                                                   'Game Mode'])])),
                ('regressor', LinearRegression())])

##Оценка на модела
Правим предсказания върху тестовия набор от данни
Изчисляваме различни метрики за оценка на точността на модела:
- MSE (средно-квадратична грешка) - по-ниски стойности са по-добри
- RMSE (корен от средно-квадратичната грешка) - в същите единици като целевата променлива
- R² (коефициент на детерминация) - показва каква част от вариацията на данните е обяснена от модела (0-1, по-високи стойности са по-добри)

In [73]:
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Evaluation:")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R-squared: {r2:.4f}")


Model Evaluation:
Mean Squared Error: 1.3404
Root Mean Squared Error: 1.1578
R-squared: 0.9769


##Визуализация на предсказанията
- Създаваме диаграма, която сравнява действителните рейтинги с предсказаните
- Идеалните предсказания биха лежали на червената пунктирана линия
- Това ни помага визуално да оценим колко добре работи моделът

In [74]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel(f'Actual {target_column}')
plt.ylabel(f'Predicted {target_column}')
plt.title('Actual vs Predicted Ratings')
plt.savefig('actual_vs_predicted.png')
plt.close()

##Анализ на остатъците
- Изчисляваме остатъците (разликата между действителните и предсказани стойности)
- Визуализираме разпределението на остатъците - в идеалния случай те трябва да имат нормално разпределение около нулата
- Създаваме диаграма на остатъците спрямо предсказаните стойности - в идеалния случай не трябва да се наблюдават систематични модели

In [75]:
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.xlabel('Residuals')
plt.title('Residual Distribution')
plt.savefig('residual_distribution.png')
plt.close()

plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.7)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel(f'Predicted {target_column}')
plt.ylabel('Residuals')
plt.title('Residuals vs Predicted Values')
plt.savefig('residuals_vs_predicted.png')
plt.close()

##Анализ на важността на характеристиките
- Извличаме и анализираме коефициентите на линейния регресионен модел
- Коефициентите показват относителната важност на всяка характеристика за предсказването на рейтингите
- Извеждаме топ 10 характеристики с най-голямо абсолютно влияние

In [76]:
if len(numeric_features) > 0:
    try:
        regressor = model.named_steps['regressor']
        preprocessor = model.named_steps['preprocessor']

        # Get all feature names (numeric + categorical)
        feature_names = []
        if len(numeric_features) > 0:
            feature_names.extend(numeric_features)
        if len(categorical_features) > 0:
            # Get transformed feature names including one-hot encoding
            cat_features = preprocessor.transformers_[1][1].get_feature_names_out(categorical_features)
            feature_names.extend(cat_features)

        # Print top coefficients with actual feature names
        print("\nTop coefficients with feature names:")
        coef = regressor.coef_
        # Sort indices by coefficient magnitude
        sorted_indices = np.argsort(np.abs(coef))[-10:]
        for idx in sorted_indices:
            if idx < len(feature_names):
                print(f"Feature {feature_names[idx]}: {coef[idx]:.4f}")
            else:
                print(f"Feature index {idx} (out of range): {coef[idx]:.4f}")

    except Exception as e:
        print(f"Could not extract feature importance: {e}")


Top coefficients with feature names:
Feature Platform_PC: 0.0221
Feature Genre_Shooter: -0.0249
Feature User Review Text_Great game, but the gameplay is amazing.: 0.0259
Feature User Review Text_Solid game, but the graphics could be better.: -0.0273
Feature Genre_Sports: 0.0277
Feature User Review Text_Solid game, but the gameplay is amazing.: 0.0312
Feature User Review Text_Great game, but too many bugs.: -0.0327
Feature Genre_RPG: 0.0463
Feature Game Length (Hours): 4.7577
Feature Price: 5.7544


##Запазване на модела
-Запазваме обучения модел във файл с помощта на библиотеката joblib
-Това позволява по-късно да използваме модела за нови предсказания, без да се налага да го обучаваме отново

In [77]:
import joblib
joblib.dump(model, 'video_game_rating_model.pkl')
print("\nModel saved as 'video_game_rating_model.pkl'")


Model saved as 'video_game_rating_model.pkl'


##Обобщение
Този код създава пълен работен процес за машинно обучение:

Зареждаме и анализираме данните за видео игри
Идентифицираме целевата променлива (рейтинги на игрите)
Предварително обработваме и трансформираме данните
Обучаваме линеен регресионен модел
Оценяваме точността на модела с различни метрики
Визуализираме резултатите и анализираме грешките
Запазваме модела за бъдеща употреба

Крайният резултат е модел, който може да предскаже потребителския рейтинг на видео игра въз основа на различни характеристики като жанр, платформа, графика, звук и други фактори, налични в набора от данни.
R² (coefficient of determination)

In [80]:
score = model.score(X_test, y_test)
print(f"Model R² score: {score:.4f}")

Model R² score: 0.9769
